In [1]:
%load_ext autoreload
%autoreload 2

In [71]:
import celestializer as cl
import numpy as np

In [72]:

img_sky = cl.RawImage(cl.Paths.observations / "2024-11-05/IMG_3320.CR2").load()
img = img_sky.raw()
img_sky.metadata.model_dump()

{'width': 5202,
 'height': 3465,
 'focal_length': 18.0,
 'aperture': 3.5,
 'shutter': 10.0,
 'iso': 6400,
 'timestamp': datetime.datetime(2024, 11, 5, 0, 37, 58),
 'camera': 'Canon EOS 1200D',
 'bit_depth': 14}

In [73]:
img_in = img.green_interpolated()
img_in = cl.correct_vignette(img_in)
img_in = img_in.channel("green")

In [ ]:
mask_hot, mask_dead = cl.find_defects(img_in)
print(f"Found {mask_hot.sum()} hot pixels and {mask_dead.sum()} dead pixels")

mask_out = np.zeros((*img_in.shape, 3), dtype=np.float32)
mask_out[..., 1] = mask_hot.astype(np.float32)
mask_out[..., 0] = mask_dead.astype(np.float32)
mask_out = cl.ImageNBit(mask_out, 1)
mask_out.save(cl.Paths.saved / "defects_mask.png")
mask = mask_hot | mask_dead
img_out = cl.replace_defects(img_in, mask)
img_out.save(cl.Paths.saved / "defects_replaced.png")

Found 42 hot pixels and 107 dead pixels
